In [1]:
import os
from llama_index.llms.perplexity import Perplexity
from dotenv import load_dotenv

load_dotenv()

pplx_api_key = os.getenv("PERPLEXITY_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_KEY')

llm = Perplexity(
    api_key=pplx_api_key, model="sonar-small-online", temperature=0.5
)

In [2]:
from llama_index.graph_stores.neo4j import Neo4jGraphStore
from llama_index.core import StorageContext

username = "neo4j"
password = os.getenv('NEO4J_PW')
url = os.getenv('NEO4J_URL')
database = "neo4j"

graph_store = Neo4jGraphStore(
    username=username,
    password=password,
    url=url,
    database=database,
)

storage_context = StorageContext.from_defaults(graph_store=graph_store)


In [3]:
from llama_index.core.indices.knowledge_graph.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from entity_extraction.entity_extraction import custom_entity_extract_fn

graph_rag_retriever_with_nl2graphquery = KnowledgeGraphRAGRetriever(
    storage_context=storage_context,
    verbose=True,
    # llm=llm,
    # entity_extract_fn=custom_entity_extract_fn,
    retriever_mode="keyword",
    with_nl2graphquery=True,
)

query_engine_with_nl2graphquery = RetrieverQueryEngine.from_args(
    graph_rag_retriever_with_nl2graphquery,
)

In [4]:
from llama_index.core.llms import ChatMessage

query = "tell me about dallas"

pre_response = query_engine_with_nl2graphquery.query(
    query,
)

if pre_response.metadata is None:
    # 1) answer needs to be queried from Llama2 endpoint
    messages_dict = [
        {"role": "system", "content": "Be precise and concise."},
        {"role": "user", "content": query},
    ]
    messages = [ChatMessage(**msg) for msg in messages_dict]

    external_response = llm.chat(messages)
    print(external_response)

    # 2) answer needs to be stored into txt file for loading into KG
    with open('data/external_response.txt', 'w') as f:
        print(external_response, file=f)
else:
    print("From Graph: " + str(pre_response))
    print(list(list(pre_response.metadata.values())[0]["kg_rel_map"].keys()))

assistant: Dallas is the third-largest city in Texas and the ninth-largest in the United States. It is known for its diverse economy, cultural attractions, and strong business climate. The city is home to many Fortune 500 companies and serves as a hub for transportation, finance, and technology industries. Dallas also hosts numerous events throughout the year, including the State Fair of Texas, the Dallas International Film Festival, and the Cotton Bowl Classic football game.


In [5]:
from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=['data/external_response.txt']
).load_data()

index = KnowledgeGraphIndex.from_documents(
    documents,
    storage_context=storage_context,
    max_triplets_per_chunk=5,
)

In [6]:
post_response = query_engine_with_nl2graphquery.query(
    query,
)

print(post_response)
print(list(list(post_response.metadata.values())[0]["kg_rel_map"].keys()))

Dallas is known for its strong business climate, cultural attractions, and diverse economy. It is the ninth-largest city in the United States and the third-largest city in Texas.
['Dallas']
